**verarbeitung von annotationen für channelislands camera traps datensatz**

herunterladen des datensatzes

In [ ]:
%%shell

wget -O azcopy_linux.tar.gz https://aka.ms/downloadazcopy-v10-linux
tar -xvzf azcopy_linux.tar.gz --wildcards */azcopy --strip 1
rm azcopy_linux.tar.gz
chmod u+x azcopy

In [ ]:
!mkdir dataset

In [ ]:
!./azcopy cp "https://lilablobssc.blob.core.windows.net/channel-islands-camera-traps/images" "/content/dataset" --recursive

vorraussetzungen für weiteres verarbeiten installieren und initialisiern

In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [ ]:
!git clone https://github.com/cocodataset/cocoapi

!cd /content/cocoapi/PythonAPI && make

In [ ]:
 !pip install timm
 
 !git clone https://github.com/Moldazien/BA.git

In [ ]:
import os
os.chdir('/content/BA')

In [ ]:
import os
import sys
import argparse
import cv2
import random
import colorsys
import requests
from io import BytesIO

import skimage.io
from skimage.measure import find_contours
import matplotlib.pyplot as plt
from matplotlib.patches import Polygon
import torch
import torch.nn as nn
import torchvision
from torchvision import transforms as pth_transforms
import numpy as np
from PIL import Image

import utils
import vision_transformer as vits

from pycocotools.coco import COCO

In [ ]:
dataset_path  = '/content/dataset'
annotations = '/content/gdrive/MyDrive/Datasets/channelisland/annotations/channel_islands_camera_traps.json' #originale annotationsdatei

datensatz auf gewünschte klassen zuschneiden + leere bilder entfernen

In [ ]:
coco = COCO(annotations)

In [ ]:
categories = coco.getCatIds()
categories = coco.loadCats(categories)

In [ ]:
cat_mapping = {'fox':1, 'skunk':2, 'rodent':3, 'bird':4} #künftiges mapping für die 4 klassen 

In [ ]:
annotations = coco.getAnnIds()
annotations = coco.loadAnns(annotations)

In [ ]:
images = coco.getImgIds()
images = coco.loadImgs(images)

In [ ]:
counter = 0

single_images = []

for img in images:
  img_id = img['id']

  ann_ids = coco.getAnnIds([img_id])

  dec = False

  if len(ann_ids) > 1:

    anns = coco.loadAnns(ann_ids)
    temp = anns[0]['category_id']
    for ann in anns:
      if ann['category_id'] != temp:
        dec = True

  if dec:
    counter = counter + 1
  
  if not dec:
    single_images.append(img)

print(counter)
print(len(single_images))
print(len(images))

In [ ]:
new_annotations = []
new_images = []
new_categories = []

In [ ]:
for img in single_images:
  img_id = img['id']

  ann_ids = coco.getAnnIds([img_id])
  anns = coco.loadAnns(ann_ids)

  #print(len(ann_ids))

  append = False
  for ann in anns:
    if ann['category_id'] in [2,3,4,5]:
      append = True
      #ann['category_id'] = ann['category_id']
      new_annotations.append(ann)

  if append:
    new_images.append(img)
  

In [ ]:
for ann in new_annotations:
  cat_id = ann['category_id']
  cate = coco.loadCats([cat_id])[0]
  ann['category_id'] = cat_mapping[cate['name']]

In [ ]:
for cat in categories:
  if cat['id'] in [2,3,4,5]:
    #cat['id'] = cat['id']#cat_mapping[cat['name']]
    new_categories.append(cat)

In [ ]:
for cat in new_categories:
  cat['id'] = cat_mapping[cat['name']]

In [ ]:
import json 

output_file = '/Speicherpfad' #annotationsdatei speichern

with open(output_file, 'w') as file:
  json.dump({'images': new_images,'categories': new_categories , 'annotations': new_annotations}, file, indent = 4, sort_keys=False)

In [ ]:
drive.flush_and_unmount()

zufallsbilder aus bestimmten klassen laden und speichern (u.a. für beispielbilder in arbeit)





In [ ]:
annotation = 'ANNOTATONS_PFAD'
coco = COCO(annotation)

In [ ]:
image_ids = coco.getImgIds()

In [ ]:
import random
random.shuffle(image_ids)

In [ ]:
new_images = []

counter1 = 3
counter2 = 3
counter3 = 3
counter4 = 3

for id in image_ids:
  image = coco.loadImgs([id])

  ann_ids = coco.getAnnIds([id])
  anns = coco.loadAnns(ann_ids)

  if anns[0]['category_id'] == 1 and counter1 > 0:
    new_images.append(image[0])
    counter1 = counter1 - 1
  if anns[0]['category_id'] == 2 and counter2 > 0:
    new_images.append(image[0])
    counter2 = counter2 - 1
  if anns[0]['category_id'] == 3 and counter3 > 0:
    new_images.append(image[0])
    counter3 = counter3 - 1
  if anns[0]['category_id'] == 4 and counter4 > 0:
    new_images.append(image[0])
    counter4 = counter4 - 1

In [ ]:
!cd /content/gdrive/MyDrive && mkdir ManualAnn

In [ ]:
#heruntergeladene bilder kopieren und in googledrive speichern

import shutil
import json

path_to_direc = '/content/dataset/images'
other_path = '/content/gdrive/MyDrive/ManualAnn'
file_type = '.jpg'

counter = 0

for item in new_images:

  oldpath = path_to_direc + '/' + item['file_name']

  newpath = other_path + '/' + str(counter) + 'I.jpg'

  counter = counter + 1 

  shutil.copyfile(oldpath, newpath)

In [ ]:
drive.flush_and_unmount()

**erstellen einer gesamten annotationsdatei aus ergebnissen des Megadetectors und der originalen annotationsdatei**

In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [ ]:
!git clone https://github.com/cocodataset/cocoapi

!cd /content/cocoapi/PythonAPI && make

In [ ]:
import os
import sys
import argparse
import cv2
import random
import colorsys
import requests
from io import BytesIO

import skimage.io
#from skimage.measure import find_contours
import matplotlib.pyplot as plt
from matplotlib.patches import Polygon
import torch
import torch.nn as nn
import torchvision
from torchvision import transforms as pth_transforms
import numpy as np
from PIL import Image

from pycocotools.coco import COCO
import json

In [ ]:
ann_file = 'ORIGINAL_ANNOTATIONEN'

megadetector_path = 'MEGADETECTOR_OUTPUT_FILE'

coco = COCO(ann_file)

with open(megadetector_path) as file:
  m_detections = json.load(file)

In [ ]:
org_images = coco.getImgIds()
org_images = coco.loadImgs(org_images)

In [ ]:
image_detections = m_detections['images']

threshold = 0.9 #gewünschter threshold für confidence der megadetector detektionen (hier noch für iNat19)

org_images = coco.getImgIds()
org_images = coco.loadImgs(org_images)

categories = coco.getCatIds()
categories = coco.loadCats(categories)

org_anns = coco.getAnnIds(coco.getImgIds())
org_anns = coco.loadAnns(org_anns)

images = []
annotations = []

id_counter = 0

counter = 0
for img in org_images:
  counter = counter + 1
  height = img['height']
  width = img['width']
  img_id = img['id']
  file_name = img['file_name']

  detect = [det for det in image_detections if det['file'].split('/images/')[1] == file_name]
  detect = [det['detections'] for det in detect]
  img_added = False
  if len(detect) >= 1:
    for item in detect:
      if len(item) >= 1:
        if item[0]['category'] == '1' and item[0]['conf'] >= threshold:
          bbox = item[0]['bbox']
          bbox = [bbox[0]*width, bbox[1]*height, bbox[2]*width, bbox[3]*height]
          score = item[0]['conf']
          area = area = bbox[2]*bbox[3]
          ann_ids = coco.getAnnIds([img_id])
          anns = coco.loadAnns(ann_ids)
          if len(anns) >= 1:
            if img_added == False:
              images.append(img)
              img_added = True

            category_id = anns[0]['category_id']
            #segmentation = anns[0]['segmentation']
            image_id = img_id
            id = id_counter
            id_counter = id_counter + 1

            annotation = {}
            annotation['image_id'] = image_id
            annotation['bbox'] = bbox
            annotation['area'] = area
            annotation['category_id'] = category_id
            annotation['score'] = score
            annotation['id'] = id

            annotations.append(annotation)

In [ ]:
#speichern der generierten Daten

import json 

output_file = '/PATH_TO_NEW_JSON'

with open(output_file, 'w') as file:
  json.dump({'images': images,'categories': categories , 'annotations': annotations}, file, indent = 4, sort_keys=False)

In [ ]:
drive.flush_and_unmount()